In [2]:
import pandas as pd
import allel
import os

In [3]:
strain=['A_J','AKR_J','BALB_CJ','CBA_J','C3H_HeJ','DBA_2J','LP_J']

tools=['BioGraph*',
'breakdancer',
'clever',
'delly',
'gasv',
'gridss',
'indelminer',
'lumpexpress',
'mistrvar',
'pindel',
'popdel',
'rdxplorer',
'smoove',
'sniffles']

In [4]:
%store -r
pdList = [df_10,df_100,df_1000,df_10000,df_10_nondel,df_100_nondel,df_1000_nondel,df_10000_nondel] 
df_cat = pd.concat(pdList)

In [5]:
df_cat.head()

,tool,strain,length,flag,position,threshold
0,sniffles,LP_J,1725,FP,6055203,10
1,sniffles,LP_J,34582728,FP,17098523,10
2,sniffles,LP_J,48,FP,17201451,10
3,sniffles,LP_J,114,FP,17231867,10
4,sniffles,LP_J,35,FP,17291187,10


# 50-100 bp Deletions

In [6]:
df_cat_50_100=df_cat.loc[(df_cat['length'] >= 50) & (df_cat['length'] <100)]
df_cat_50_100.head()

,tool,strain,length,flag,position,threshold
23,sniffles,LP_J,68,FP,22431473,10
27,sniffles,LP_J,73,FP,23210349,10
32,sniffles,LP_J,52,FP,25674538,10
37,sniffles,LP_J,52,FP,27784186,10
41,sniffles,LP_J,84,FP,29442320,10


In [7]:
group_TP_50_100 = df_cat_50_100[df_cat_50_100['flag'] == 'TP'].groupby(['tool','threshold','strain'],as_index=False)['flag'].count()
group_TP_50_100 = group_TP_50_100.rename(columns={"flag": "nTP"})
group_TP_50_100.shape
group_TP_50_100.head()

,tool,threshold,strain,nTP
0,BioGraph*,10,AKR_J,2
1,BioGraph*,10,A_J,2
2,BioGraph*,10,BALB_CJ,1
3,BioGraph*,10,C3H_HeJ,2
4,BioGraph*,10,CBA_J,3


In [8]:
group_FP_50_100=df_cat_50_100[df_cat_50_100['flag'] == 'FP'].groupby(['tool','threshold','strain'],as_index=False)['flag'].count()
group_FP_50_100=group_FP_50_100.rename(columns={"flag": "nFP"})
group_FP_50_100.shape
group_FP_50_100.head()

,tool,threshold,strain,nFP
0,BioGraph*,10,AKR_J,132
1,BioGraph*,10,A_J,136
2,BioGraph*,10,BALB_CJ,145
3,BioGraph*,10,C3H_HeJ,153
4,BioGraph*,10,CBA_J,169


In [9]:
group_TN_50_100=df_cat_50_100[df_cat_50_100['flag'] == 'TN'].groupby(['tool','threshold','strain'],as_index=False)['flag'].count()
group_TN_50_100=group_TN_50_100.rename(columns={"flag": "nTN"})
group_TN_50_100.shape
group_TN_50_100.head()

,tool,threshold,strain,nTN
0,delly,1000,BALB_CJ,1
1,delly,10000,AKR_J,1
2,delly,10000,BALB_CJ,1
3,gasv,10000,AKR_J,3
4,gasv,10000,A_J,1


In [10]:
for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            if not (((group_TP_50_100['tool'] == t) & (group_TP_50_100['strain'] == s) & (group_TP_50_100['threshold'] == th)).any()):
                group_TP_50_100 = group_TP_50_100.append({'tool' : t , 'strain' : s,'threshold' : th,'nTP' : 0} , ignore_index=True)
group_TP_50_100.shape

(392, 4)

In [11]:
for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            if not (((group_FP_50_100['tool'] == t) & (group_FP_50_100['strain'] == s) & (group_FP_50_100['threshold'] == th)).any()):
                group_FP_50_100 = group_FP_50_100.append({'tool' : t , 'strain' : s,'threshold' : th,'nFP' : 0} , ignore_index=True)
                #print (s,t,th)
group_FP_50_100.shape

(392, 4)

In [12]:
for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            if not (((group_TN_50_100['tool'] == t) & (group_TN_50_100['strain'] == s) & (group_TN_50_100['threshold'] == th)).any()):
                group_TN_50_100 = group_TN_50_100.append({'tool' : t , 'strain' : s,'threshold' : th,'nTN' : 0} , ignore_index=True)
                #print (s,t,th)
group_TN_50_100.shape

(392, 4)

In [13]:
df_merge_50_100=pd.merge(group_TP_50_100, group_FP_50_100,on=['tool','threshold','strain'])
df_merge_50_100=pd.merge(df_merge_50_100, group_TN_50_100,on=['tool','threshold','strain'])
df_merge_50_100.shape
df_merge_50_100.head(10)

,tool,threshold,strain,nTP,nFP,nTN
0,BioGraph*,10,AKR_J,2,132,0
1,BioGraph*,10,A_J,2,136,0
2,BioGraph*,10,BALB_CJ,1,145,0
3,BioGraph*,10,C3H_HeJ,2,153,0
4,BioGraph*,10,CBA_J,3,169,0
5,BioGraph*,10,DBA_2J,3,164,0
6,BioGraph*,10,LP_J,2,124,0
7,BioGraph*,100,AKR_J,18,116,0
8,BioGraph*,100,A_J,19,119,0
9,BioGraph*,100,BALB_CJ,19,127,0


In [14]:
#df_current = pd.DataFrame(columns=['strain','length'])
#df = pd.DataFrame(columns=['strain','length'])


#for s in strain:
    #file='../gold_standard/mouse_vcf/'+s+'_reference.vcf'
    #callset = allel.read_vcf(file,fields='*')
    
    #df_current = pd.DataFrame({'strain': s, 'length': callset['variants/SVLEN']})
    #df = pd.concat([df_current, df],ignore_index=True)
#group_data_true = df.groupby(['strain'],as_index=False).count()
#group_data_true=group_data_true.rename(columns={"length": "n_true"})

#group_data_true.head(7)

In [15]:
#df_merge['nFN']=df_merge['n_true']-df_merge['nTP']

df_merge_50_100['sensitivity']=df_merge_50_100['nTP']/(df_merge_50_100['nTP']+df_merge_50_100['nFP'])
df_merge_50_100['precision']=df_merge_50_100['nTP']/(df_merge_50_100['nTP']+df_merge_50_100['nFP'])
df_merge_50_100['specificity']=df_merge_50_100['nTN']/(df_merge_50_100['nTN']+df_merge_50_100['nFP'])
df_merge_50_100['f-score']=2*(df_merge_50_100['sensitivity']*df_merge_50_100['precision'])/(df_merge_50_100['sensitivity']+df_merge_50_100['precision']+0.00000001)
df_merge_50_100.head()

,tool,threshold,strain,nTP,nFP,nTN,sensitivity,precision,specificity,f-score
0,BioGraph*,10,AKR_J,2,132,0,0.014925,0.014925,0.0,0.014925
1,BioGraph*,10,A_J,2,136,0,0.014493,0.014493,0.0,0.014493
2,BioGraph*,10,BALB_CJ,1,145,0,0.006849,0.006849,0.0,0.006849
3,BioGraph*,10,C3H_HeJ,2,153,0,0.012903,0.012903,0.0,0.012903
4,BioGraph*,10,CBA_J,3,169,0,0.017442,0.017442,0.0,0.017442


# 100-500 bp Deletions

In [16]:
df_cat_100_500=df_cat.loc[(df_cat['length'] >= 100) & (df_cat['length'] <500)]
df_cat_100_500.head()

,tool,strain,length,flag,position,threshold
3,sniffles,LP_J,114,FP,17231867,10
11,sniffles,LP_J,341,FP,18926467,10
19,sniffles,LP_J,106,FP,21474388,10
25,sniffles,LP_J,349,FP,23059354,10
29,sniffles,LP_J,470,FP,23214172,10


In [17]:
group_TP_100_500 = df_cat_100_500[df_cat_100_500['flag'] == 'TP'].groupby(['tool','threshold','strain'],as_index=False)['flag'].count()
group_TP_100_500 = group_TP_100_500.rename(columns={"flag": "nTP"})
group_TP_100_500.shape
group_TP_100_500.head()

,tool,threshold,strain,nTP
0,BioGraph*,10,AKR_J,19
1,BioGraph*,10,A_J,29
2,BioGraph*,10,BALB_CJ,26
3,BioGraph*,10,C3H_HeJ,24
4,BioGraph*,10,CBA_J,28


In [18]:
group_FP_100_500=df_cat_100_500[df_cat_100_500['flag'] == 'FP'].groupby(['tool','threshold','strain'],as_index=False)['flag'].count()
group_FP_100_500=group_FP_100_500.rename(columns={"flag": "nFP"})
group_FP_100_500.shape
group_FP_100_500.head()

,tool,threshold,strain,nFP
0,BioGraph*,10,AKR_J,153
1,BioGraph*,10,A_J,162
2,BioGraph*,10,BALB_CJ,151
3,BioGraph*,10,C3H_HeJ,156
4,BioGraph*,10,CBA_J,169


In [19]:
group_TN_100_500=df_cat_100_500[df_cat_100_500['flag'] == 'TN'].groupby(['tool','threshold','strain'],as_index=False)['flag'].count()
group_TN_100_500=group_TN_100_500.rename(columns={"flag": "nTN"})
group_TN_100_500.shape
group_TN_100_500.head()

,tool,threshold,strain,nTN
0,BioGraph*,10000,AKR_J,1
1,BioGraph*,10000,A_J,1
2,BioGraph*,10000,BALB_CJ,1
3,BioGraph*,10000,C3H_HeJ,1
4,BioGraph*,10000,CBA_J,1


In [20]:
for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            if not (((group_TP_100_500['tool'] == t) & (group_TP_100_500['strain'] == s) & (group_TP_100_500['threshold'] == th)).any()):
                group_TP_100_500 = group_TP_100_500.append({'tool' : t , 'strain' : s,'threshold' : th,'nTP' : 0} , ignore_index=True)
group_TP_100_500.shape

(392, 4)

In [21]:
for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            if not (((group_FP_100_500['tool'] == t) & (group_FP_100_500['strain'] == s) & (group_FP_100_500['threshold'] == th)).any()):
                group_FP_100_500 = group_FP_100_500.append({'tool' : t , 'strain' : s,'threshold' : th,'nFP' : 0} , ignore_index=True)
group_FP_100_500.shape

(392, 4)

In [22]:
for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            if not (((group_TN_100_500['tool'] == t) & (group_TN_100_500['strain'] == s) & (group_TN_100_500['threshold'] == th)).any()):
                group_TN_100_500 = group_TN_100_500.append({'tool' : t , 'strain' : s,'threshold' : th,'nTN' : 0} , ignore_index=True)
group_TN_100_500.shape

(392, 4)

In [23]:
df_merge_100_500=pd.merge(group_TP_100_500, group_FP_100_500,on=['tool','threshold','strain'])
df_merge_100_500=pd.merge(df_merge_100_500, group_TN_100_500,on=['tool','threshold','strain'])
df_merge_100_500.shape

(392, 6)

In [24]:
df_merge_100_500.head(10)

,tool,threshold,strain,nTP,nFP,nTN
0,BioGraph*,10,AKR_J,19,153,0
1,BioGraph*,10,A_J,29,162,0
2,BioGraph*,10,BALB_CJ,26,151,0
3,BioGraph*,10,C3H_HeJ,24,156,0
4,BioGraph*,10,CBA_J,28,169,0
5,BioGraph*,10,DBA_2J,28,173,0
6,BioGraph*,10,LP_J,19,118,0
7,BioGraph*,100,AKR_J,131,41,0
8,BioGraph*,100,A_J,147,44,0
9,BioGraph*,100,BALB_CJ,140,37,0


In [25]:
df_merge_100_500['sensitivity']=df_merge_100_500['nTP']/(df_merge_100_500['nTP']+df_merge_100_500['nFP'])
df_merge_100_500['precision']=df_merge_100_500['nTP']/(df_merge_100_500['nTP']+df_merge_100_500['nFP'])
df_merge_100_500['specificity']=df_merge_100_500['nTN']/(df_merge_100_500['nTN']+df_merge_100_500['nFP'])
df_merge_100_500['f-score']=2*(df_merge_100_500['sensitivity']*df_merge_100_500['precision'])/(df_merge_100_500['sensitivity']+df_merge_100_500['precision']+0.00000001)
df_merge_100_500.head()

,tool,threshold,strain,nTP,nFP,nTN,sensitivity,precision,specificity,f-score
0,BioGraph*,10,AKR_J,19,153,0,0.110465,0.110465,0.0,0.110465
1,BioGraph*,10,A_J,29,162,0,0.151832,0.151832,0.0,0.151832
2,BioGraph*,10,BALB_CJ,26,151,0,0.146893,0.146893,0.0,0.146893
3,BioGraph*,10,C3H_HeJ,24,156,0,0.133333,0.133333,0.0,0.133333
4,BioGraph*,10,CBA_J,28,169,0,0.142132,0.142132,0.0,0.142132


# 500-1000 bp Deletions

In [26]:
df_cat_500_1000=df_cat.loc[(df_cat['length'] >= 500) & (df_cat['length'] <1000)]
df_cat_500_1000.head()

,tool,strain,length,flag,position,threshold
9,sniffles,LP_J,618,FP,18396654,10
33,sniffles,LP_J,709,FP,25931984,10
51,sniffles,LP_J,524,FP,31997429,10
53,sniffles,LP_J,777,FP,36762558,10
74,sniffles,LP_J,790,FP,43239099,10


In [27]:
group_TP_500_1000 = df_cat_500_1000[df_cat_500_1000['flag'] == 'TP'].groupby(['tool','threshold','strain'],as_index=False)['flag'].count()
group_TP_500_1000 = group_TP_500_1000.rename(columns={"flag": "nTP"})
group_TP_500_1000.shape
group_TP_500_1000.head()

,tool,threshold,strain,nTP
0,BioGraph*,10,AKR_J,5
1,BioGraph*,10,A_J,3
2,BioGraph*,10,BALB_CJ,4
3,BioGraph*,10,C3H_HeJ,4
4,BioGraph*,10,CBA_J,4


In [28]:
group_FP_500_1000=df_cat_500_1000[df_cat_500_1000['flag'] == 'FP'].groupby(['tool','threshold','strain'],as_index=False)['flag'].count()
group_FP_500_1000=group_FP_500_1000.rename(columns={"flag": "nFP"})
group_FP_500_1000.shape
group_FP_500_1000.head()

,tool,threshold,strain,nFP
0,BioGraph*,10,AKR_J,27
1,BioGraph*,10,A_J,29
2,BioGraph*,10,BALB_CJ,26
3,BioGraph*,10,C3H_HeJ,26
4,BioGraph*,10,CBA_J,25


In [29]:
group_TN_500_1000=df_cat_500_1000[df_cat_500_1000['flag'] == 'TN'].groupby(['tool','threshold','strain'],as_index=False)['flag'].count()
group_TN_500_1000=group_TN_500_1000.rename(columns={"flag": "nTN"})
group_TN_500_1000.shape
group_TN_500_1000.head()

,tool,threshold,strain,nTN
0,BioGraph*,100,AKR_J,3
1,BioGraph*,100,A_J,2
2,BioGraph*,100,BALB_CJ,2
3,BioGraph*,100,C3H_HeJ,3
4,BioGraph*,100,CBA_J,3


In [30]:
for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            if not (((group_TP_500_1000['tool'] == t) & (group_TP_500_1000['strain'] == s) & (group_TP_500_1000['threshold'] == th)).any()):
                group_TP_500_1000 = group_TP_500_1000.append({'tool' : t , 'strain' : s,'threshold' : th,'nTP' : 0} , ignore_index=True)
group_TP_500_1000.shape

(392, 4)

In [31]:
for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            if not (((group_FP_500_1000['tool'] == t) & (group_FP_500_1000['strain'] == s) & (group_FP_500_1000['threshold'] == th)).any()):
                group_FP_500_1000 = group_FP_500_1000.append({'tool' : t , 'strain' : s,'threshold' : th,'nFP' : 0} , ignore_index=True)
group_FP_500_1000.shape

(392, 4)

In [32]:
for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            if not (((group_TN_500_1000['tool'] == t) & (group_TN_500_1000['strain'] == s) & (group_TN_500_1000['threshold'] == th)).any()):
                group_TN_500_1000 = group_TN_500_1000.append({'tool' : t , 'strain' : s,'threshold' : th,'nTN' : 0} , ignore_index=True)
group_TN_500_1000.shape

(392, 4)

In [33]:
df_merge_500_1000=pd.merge(group_TP_500_1000, group_FP_500_1000,on=['tool','threshold','strain'])
df_merge_500_1000=pd.merge(df_merge_500_1000, group_TN_500_1000,on=['tool','threshold','strain'])
df_merge_500_1000.shape

(392, 6)

In [34]:
df_merge_500_1000.head(10)

,tool,threshold,strain,nTP,nFP,nTN
0,BioGraph*,10,AKR_J,5,27,0
1,BioGraph*,10,A_J,3,29,0
2,BioGraph*,10,BALB_CJ,4,26,0
3,BioGraph*,10,C3H_HeJ,4,26,0
4,BioGraph*,10,CBA_J,4,25,0
5,BioGraph*,10,DBA_2J,3,27,0
6,BioGraph*,10,LP_J,3,19,0
7,BioGraph*,100,AKR_J,28,4,3
8,BioGraph*,100,A_J,26,6,2
9,BioGraph*,100,BALB_CJ,25,5,2


In [35]:
df_merge_500_1000['sensitivity']=df_merge_500_1000['nTP']/(df_merge_500_1000['nTP']+df_merge_500_1000['nFP'])
df_merge_500_1000['precision']=df_merge_500_1000['nTP']/(df_merge_500_1000['nTP']+df_merge_500_1000['nFP'])
df_merge_500_1000['specificity']=df_merge_500_1000['nTN']/(df_merge_500_1000['nTN']+df_merge_500_1000['nFP'])
df_merge_500_1000['f-score']=2*(df_merge_500_1000['sensitivity']*df_merge_500_1000['precision'])/(df_merge_500_1000['sensitivity']+df_merge_500_1000['precision']+0.00000001)
df_merge_500_1000.head()

,tool,threshold,strain,nTP,nFP,nTN,sensitivity,precision,specificity,f-score
0,BioGraph*,10,AKR_J,5,27,0,0.156250,0.156250,0.0,0.156250
1,BioGraph*,10,A_J,3,29,0,0.093750,0.093750,0.0,0.093750
2,BioGraph*,10,BALB_CJ,4,26,0,0.133333,0.133333,0.0,0.133333
3,BioGraph*,10,C3H_HeJ,4,26,0,0.133333,0.133333,0.0,0.133333
4,BioGraph*,10,CBA_J,4,25,0,0.137931,0.137931,0.0,0.137931


# 1000+ bp Deletions

In [36]:
df_cat_1000=df_cat.loc[(df_cat['length'] >= 1000)]
df_cat_1000.head()

,tool,strain,length,flag,position,threshold
0,sniffles,LP_J,1725,FP,6055203,10
1,sniffles,LP_J,34582728,FP,17098523,10
7,sniffles,LP_J,2168,FP,18195112,10
10,sniffles,LP_J,4150,FP,18920153,10
18,sniffles,LP_J,1148,FP,21448557,10


In [37]:
group_TP_1000 = df_cat_1000[df_cat_1000['flag'] == 'TP'].groupby(['tool','threshold','strain'],as_index=False)['flag'].count()
group_TP_1000 = group_TP_1000.rename(columns={"flag": "nTP"})
group_TP_1000.shape
group_TP_1000.head()

,tool,threshold,strain,nTP
0,BioGraph*,10,AKR_J,10
1,BioGraph*,10,A_J,13
2,BioGraph*,10,BALB_CJ,14
3,BioGraph*,10,C3H_HeJ,11
4,BioGraph*,10,CBA_J,9


In [38]:
group_FP_1000 = df_cat_1000[df_cat_1000['flag'] == 'FP'].groupby(['tool','threshold','strain'],as_index=False)['flag'].count()
group_FP_1000 = group_FP_1000.rename(columns={"flag": "nFP"})
group_FP_1000.shape
group_FP_1000.head()

,tool,threshold,strain,nFP
0,BioGraph*,10,AKR_J,77
1,BioGraph*,10,A_J,94
2,BioGraph*,10,BALB_CJ,95
3,BioGraph*,10,C3H_HeJ,81
4,BioGraph*,10,CBA_J,82


In [39]:
group_TN_1000 = df_cat_1000[df_cat_1000['flag'] == 'TN'].groupby(['tool','threshold','strain'],as_index=False)['flag'].count()
group_TN_1000 = group_TN_1000.rename(columns={"flag": "nTN"})
group_TN_1000.shape
group_TN_1000.head()

,tool,threshold,strain,nTN
0,BioGraph*,10,AKR_J,11
1,BioGraph*,10,A_J,20
2,BioGraph*,10,BALB_CJ,13
3,BioGraph*,10,C3H_HeJ,17
4,BioGraph*,10,CBA_J,18


In [40]:
for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            if not (((group_TP_1000['tool'] == t) & (group_TP_1000['strain'] == s) & (group_TP_1000['threshold'] == th)).any()):
                group_TP_1000 = group_TP_1000.append({'tool' : t , 'strain' : s,'threshold' : th,'nTP' : 0} , ignore_index=True)
group_TP_1000.shape

(392, 4)

In [41]:
for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            if not (((group_FP_1000['tool'] == t) & (group_FP_1000['strain'] == s) & (group_FP_1000['threshold'] == th)).any()):
                group_FP_1000 = group_FP_1000.append({'tool' : t , 'strain' : s,'threshold' : th,'nFP' : 0} , ignore_index=True)
group_FP_1000.shape

(392, 4)

In [42]:
for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            if not (((group_TN_1000['tool'] == t) & (group_TN_1000['strain'] == s) & (group_TN_1000['threshold'] == th)).any()):
                group_TN_1000 = group_TN_1000.append({'tool' : t , 'strain' : s,'threshold' : th,'nTN' : 0} , ignore_index=True)
group_TN_1000.shape

(392, 4)

In [43]:
df_merge_1000=pd.merge(group_TP_1000, group_FP_1000,on=['tool','threshold','strain'])
df_merge_1000=pd.merge(df_merge_1000, group_TN_1000,on=['tool','threshold','strain'])
df_merge_1000.shape

(392, 6)

In [44]:
df_merge_500_1000.head(10)

,tool,threshold,strain,nTP,nFP,nTN,sensitivity,precision,specificity,f-score
0,BioGraph*,10,AKR_J,5,27,0,0.156250,0.156250,0.000000,0.156250
1,BioGraph*,10,A_J,3,29,0,0.093750,0.093750,0.000000,0.093750
2,BioGraph*,10,BALB_CJ,4,26,0,0.133333,0.133333,0.000000,0.133333
3,BioGraph*,10,C3H_HeJ,4,26,0,0.133333,0.133333,0.000000,0.133333
4,BioGraph*,10,CBA_J,4,25,0,0.137931,0.137931,0.000000,0.137931
5,BioGraph*,10,DBA_2J,3,27,0,0.100000,0.100000,0.000000,0.100000
6,BioGraph*,10,LP_J,3,19,0,0.136364,0.136364,0.000000,0.136364
7,BioGraph*,100,AKR_J,28,4,3,0.875000,0.875000,0.428571,0.875000
8,BioGraph*,100,A_J,26,6,2,0.812500,0.812500,0.250000,0.812500
9,BioGraph*,100,BALB_CJ,25,5,2,0.833333,0.833333,0.285714,0.833333


In [45]:
df_merge_1000['sensitivity']=df_merge_1000['nTP']/(df_merge_1000['nTP']+df_merge_1000['nFP'])
df_merge_1000['precision']=df_merge_1000['nTP']/(df_merge_1000['nTP']+df_merge_1000['nFP'])
df_merge_1000['specificity']=df_merge_1000['nTN']/(df_merge_1000['nTN']+df_merge_1000['nFP'])
df_merge_1000['f-score']=2*(df_merge_1000['sensitivity']*df_merge_1000['precision'])/(df_merge_1000['sensitivity']+df_merge_1000['precision']+0.00000001)
df_merge_1000.head()

,tool,threshold,strain,nTP,nFP,nTN,sensitivity,precision,specificity,f-score
0,BioGraph*,10,AKR_J,10,77,11,0.114943,0.114943,0.125000,0.114943
1,BioGraph*,10,A_J,13,94,20,0.121495,0.121495,0.175439,0.121495
2,BioGraph*,10,BALB_CJ,14,95,13,0.128440,0.128440,0.120370,0.128440
3,BioGraph*,10,C3H_HeJ,11,81,17,0.119565,0.119565,0.173469,0.119565
4,BioGraph*,10,CBA_J,9,82,18,0.098901,0.098901,0.180000,0.098901


In [46]:
%store df_merge_50_100
%store df_merge_100_500
%store df_merge_500_1000
%store df_merge_1000

Stored 'df_merge_50_100' (DataFrame)
Stored 'df_merge_100_500' (DataFrame)
Stored 'df_merge_500_1000' (DataFrame)
Stored 'df_merge_1000' (DataFrame)
